In [1]:
## Importing libraries 
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
import numpy as np
import soundfile as sf
from pathlib import Path
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel
import IPython.display as ipd

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-07-26 09:57:04 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-26 09:57:04 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [2]:
## Importing libraries 
import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
from omegaconf import OmegaConf, open_dict
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager
from scipy.io import wavfile
import noisereduce as nr

In [16]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-07-26 15:25:12 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-07-26 15:25:12 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-07-26 15:25:12 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-26 15:25:13 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-07-26 15:25:13 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-07-26 15:25:13 features:200] PADDING: 16
[NeMo I 2022-07-26 15:25:14 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


In [6]:
spec_model = FastPitchModel.from_pretrained("tts_en_fastpitch")

[NeMo I 2022-07-20 14:39:59 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo.
[NeMo I 2022-07-20 14:39:59 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo
[NeMo I 2022-07-20 14:39:59 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-07-20 14:40:04 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-20 14:40:32 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-20 14:40:32 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_durat

[NeMo I 2022-07-20 14:40:32 features:200] PADDING: 1
[NeMo I 2022-07-20 14:40:32 save_restore_connector:243] Model FastPitchModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo.


# Retrying to finetune vocoder with pretrained spec_model

In [8]:
def load_wav(audio_file, target_sr=None):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
        sample_rate = f.samplerate
        if target_sr is not None and target_sr != sample_rate:
            samples = librosa.core.resample(samples, orig_sr=sample_rate, target_sr=target_sr)
    return samples.transpose()

# Get records from the training manifest
records = []
train_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_train.json'
with open(train_manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

beta_binomial_interpolator = BetaBinomialInterpolator()
spec_model.eval()

device = spec_model.device

save_dir = Path('/home/dgxuser/Desktop/kailin/melSpect/S91/trainSpect')
save_dir.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
    # Again, our finetuned FastPitch model doesn't use multiple speakers,
    # but we keep the code to support it here for reference
    if spec_model.fastpitch.speaker_emb is not None and "speaker" in r:
        speaker = torch.tensor([r['speaker']]).to(device)
    else:
        speaker = None
    
    with torch.no_grad():
        #if "normalized_text" in r:
            #text = spec_model.parse(r["normalized_text"], normalize=False)
        #else:
            #text = spec_model.parse(r['text'])
        
        text = spec_model.parse(r['text'])
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
          speaker=speaker,
        )[0]
        
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_train.json'
with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')

In [9]:
# Get records from the validation manifest
records = []
val_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_val.json'
with open(val_manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

beta_binomial_interpolator = BetaBinomialInterpolator()
spec_model.eval()

device = spec_model.device

save_dir = Path('/home/dgxuser/Desktop/kailin/melSpect/S91/valSpect')
save_dir.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
    # Again, our finetuned FastPitch model doesn't use multiple speakers,
    # but we keep the code to support it here for reference
    if spec_model.fastpitch.speaker_emb is not None and "speaker" in r:
        speaker = torch.tensor([r['speaker']]).to(device)
    else:
        speaker = None
    
    with torch.no_grad():
        #if "normalized_text" in r:
            #text = spec_model.parse(r["normalized_text"], normalize=False)
        #else:
            #text = spec_model.parse(r['text'])
            
        text = spec_model.parse(r['text'])
        
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
          speaker=speaker,
        )[0]
        
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_val.json'
with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')

In [10]:
!(python hifigan_finetune.py \
  --config-name=hifigan.yaml \
  model.train_ds.dataloader_params.batch_size=32 \
  model.max_steps=1000 \
  ~model.optim.sched \
  model.optim.lr=0.00001 \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_val.json \
  exp_manager.exp_dir=S91_hifigan_ft \
  +init_from_pretrained_model=tts_hifigan \
  trainer.check_val_every_n_epoch=10 \
  model/train_ds=train_ds_finetune \
  model/validation_ds=val_ds_finetune)

[NeMo W 2022-07-20 14:42:09 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-20 14:42:09 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-07-20 14:42:10 nemo_logging:349] /home/dgxuser/anaconda3/envs/test4/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'hifigan.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_co

[NeMo W 2022-07-20 14:42:23 nemo_logging:349] /home/dgxuser/anaconda3/envs/test4/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1927: PossibleUserWarning: The number of training batches (19) is smaller than the logging interval Trainer(log_every_n_steps=100). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
      rank_zero_warn(
    
Epoch 0:   0%|                                           | 0/19 [00:00<?, ?it/s][W reducer.cpp:1289] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator(

In [11]:
def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        model_name="HifiGan"
    ):    
    
    d = f"S{new_speaker_id}_hifigan_ft"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [12]:
speaker_id = 91
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id)
print(last_ckpt)
vocoder = HifiGanModel.load_from_checkpoint(last_ckpt)
vocoder = vocoder.eval().cuda()

S91_hifigan_ft/HifiGan/2022-07-20_14-42-10/checkpoints/HifiGan--val_loss=1.7281-epoch=19-last.ckpt


[NeMo W 2022-07-20 14:55:49 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_train.json
      sample_rate: 22050
      n_segments: 8192
      max_duration: null
      min_duration: 0.75
      load_precomputed_mel: true
      hop_length: 256
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 32
      num_workers: 4
      pin_memory: true
    
[NeMo W 2022-07-20 14:55:49 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collecti

[NeMo I 2022-07-20 14:55:49 features:200] PADDING: 0
[NeMo I 2022-07-20 14:55:49 features:208] STFT using exact pad
[NeMo I 2022-07-20 14:55:49 features:200] PADDING: 0
[NeMo I 2022-07-20 14:55:49 features:208] STFT using exact pad


In [11]:
def infer(spec_gen_model, vocoder_model, str_input):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=None)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        mixing_enabled, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"S{new_speaker_id}_{mixing}"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [14]:
speaker_id = 91
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

S91_no_mixing/FastPitch/2022-07-18_15-07-19/checkpoints/FastPitch--v_loss=1.7809-epoch=24-last.ckpt
[NeMo I 2022-07-20 14:55:55 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-20 14:56:22 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-20 14:56:22 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_clean_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
     

[NeMo I 2022-07-20 14:56:22 features:200] PADDING: 1


FastPitchModel(
  (mel_loss): MelLoss()
  (pitch_loss): PitchLoss()
  (duration_loss): DurationLoss()
  (aligner): AlignmentEncoder(
    (softmax): Softmax(dim=3)
    (log_softmax): LogSoftmax(dim=3)
    (key_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(384, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(768, 80, kernel_size=(1,), stride=(1,))
      )
    )
    (query_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(80, 160, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(160, 80, kernel_size=(1,), stride=(1,))
      )
      (3): ReLU()
      (4): ConvNorm(
        (conv): Conv1d(80, 80, kernel_size=(1,), stride=(1,))
      )
    )
  )
  (forward_sum_loss): ForwardSumLoss(
    (log_softmax): LogSoftmax(dim=3)
    (ctc_loss): CTCLoss()
  )
  (bin_loss): BinLoss()
  (preprocessor): AudioToMelSpectrogramPreprocessor(
  

In [15]:
from pydub import AudioSegment
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_ft/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS' + str(speaker_id) + '_ft.json'

n = 0
with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f: 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

In [16]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
error = wer(transcript, predicted)
print('Word Error Rate (FineTuned): ' + str(error))

Word Error Rate (FineTuned): 0.3217330781277188


# Testing with Multiple Speakers (3F)

In [21]:
## Finding original WER (individual speakers)
speaker_id = 124
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S124/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS124.json'

n = 0
with open(manifest_path, 'w') as fout: 
    with open(text_dir, "r") as f: 
        while True: 
          text = f.readline().strip()
          if text == '': 
            break 
          spec, audio = infer(spec_model, vocoder, text)
          audio = ipd.Audio(audio, rate=22050)
          audio = AudioSegment(audio.data)
          file_dir = audio_dir + str(n) + '.wav'
          audio.export(file_dir, format="wav")

          duration = librosa.core.get_duration(filename=file_dir)
          metadata = {
            "audio_filepath": file_dir,
            "duration": duration, 
            "text": text
          }
          json.dump(metadata, fout)
          fout.write('\n')   
          n = n + 1 
            
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()
        
error = wer(transcript, predicted)
print('Word Error Rate (Speaker 124): ' + str(error))

speaker_id = 135
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S135/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS135.json'

n = 0
with open(manifest_path, 'w') as fout: 
    with open(text_dir, "r") as f: 
        while True: 
          text = f.readline().strip()
          if text == '': 
            break 
          spec, audio = infer(spec_model, vocoder, text)
          audio = ipd.Audio(audio, rate=22050)
          audio = AudioSegment(audio.data)
          file_dir = audio_dir + str(n) + '.wav'
          audio.export(file_dir, format="wav")

          duration = librosa.core.get_duration(filename=file_dir)
          metadata = {
            "audio_filepath": file_dir,
            "duration": duration, 
            "text": text
          }
          json.dump(metadata, fout)
          fout.write('\n')   
          n = n + 1 
            
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()
        
error = wer(transcript, predicted)
print('Word Error Rate (Speaker 135): ' + str(error))

S124_no_mixing/FastPitch/2022-07-07_16-30-35/checkpoints/FastPitch--v_loss=2.6331-epoch=49-last.ckpt
[NeMo I 2022-07-21 15:03:49 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-21 15:04:23 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-21 15:04:23 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S124_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_

[NeMo I 2022-07-21 15:04:23 features:200] PADDING: 1
[NeMo I 2022-07-21 15:04:23 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-21 15:04:23 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-21 15:04:23 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-21 15:04:27 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-21 15:04:27 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-21 15:04:27 features:200] PADDING: 0


...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Word Error Rate (Speaker 135): 0.1626935792587437


In [7]:
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_124_135_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_124_135_val.json \
  sup_data_path=./fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.01 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=./S91_124_135_no_mixing \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  trainer.check_val_every_n_epoch=20 \
  model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 \
  model.n_speakers=1 model.pitch_mean=121.9 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

[NeMo W 2022-07-26 10:37:07 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-26 10:37:08 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-07-26 10:37:09 exp_manager:287] Experiments will be logged at

Epoch 19: 100%|██████████| 58/58 [00:36<00:00,  1.60it/s, loss=4.95, v_num=7-09]
                                                                                Epoch 19, global step 1100: 'v_loss' reached 2.09054 (best 2.09054), saving model to '/home/dgxuser/Desktop/kailin/S91_124_135_no_mixing/FastPitch/2022-07-26_10-37-09/checkpoints/FastPitch--v_loss=2.0905-epoch=19.ckpt' as top 3
Epoch 39:  95%|█████████▍| 55/58 [00:28<00:01,  1.94it/s, loss=4.58, v_num=7-09]
Validation: 0it [00:00, ?it/s]
Epoch 39: 100%|██████████| 58/58 [00:36<00:00,  1.58it/s, loss=4.58, v_num=7-09]
                                                                                Epoch 39, global step 2200: 'v_loss' reached 2.38012 (best 2.09054), saving model to '/home/dgxuser/Desktop/kailin/S91_124_135_no_mixing/FastPitch/2022-07-26_10-37-09/checkpoints/FastPitch--v_loss=2.3801-epoch=39.ckpt' as top 3
Epoch 59:  95%|█████████▍| 55/58 [00:26<00:01,  2.08it/s, loss=4.53, v_num=7-09]
Validation: 0it [00:00, ?it/s

In [17]:
from pydub import AudioSegment
speaker_id = '91_124_135'
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_124_135/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS91_124_135.json'

n = 0
with open(manifest_path, 'w') as fout: 
    with open(text_dir, "r") as f: 
        while True: 
          text = f.readline().strip()
          if text == '': 
            break 
          spec, audio = infer(spec_model, vocoder, text)
          audio = ipd.Audio(audio, rate=22050)
          audio = AudioSegment(audio.data)
          file_dir = audio_dir + str(n) + '.wav'
          audio.export(file_dir, format="wav")

          duration = librosa.core.get_duration(filename=file_dir)
          metadata = {
            "audio_filepath": file_dir,
            "duration": duration, 
            "text": text
          }
          json.dump(metadata, fout)
          fout.write('\n')   
          n = n + 1 
            
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()
        
error = wer(transcript, predicted)
print('Word Error Rate (Combined): ' + str(error))

S91_124_135_no_mixing/FastPitch/2022-07-26_10-37-09/checkpoints/FastPitch--v_loss=2.2690-epoch=179-last.ckpt
[NeMo I 2022-07-26 15:25:25 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-26 15:25:56 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-26 15:25:56 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_124_135_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
   

[NeMo I 2022-07-26 15:25:56 features:200] PADDING: 1
[NeMo I 2022-07-26 15:25:57 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-26 15:25:57 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-26 15:25:57 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-26 15:26:00 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-26 15:26:00 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-26 15:26:00 features:200] PADDING: 0


...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Word Error Rate (Combined): 0.15451539933878544


# Reducing Pitch Peaks 

In [38]:
# For original audio training
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_transposed_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_transposed_val.json \
  sup_data_path=./fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.01 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=./S91_no_mixing_transposed \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  +trainer.max_steps=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=25 \
  model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 \
  model.n_speakers=1 model.pitch_mean=121.9 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

/home/dgxuser/anaconda3/envs/test4/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
[NeMo W 2022-07-27 10:14:26 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-27 10:14:27 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
########################################

Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|██████████| 25/25 [00:17<00:00,  1.46it/s, loss=4.28, v_num=4-28]
                                                                                Epoch 24, global step 600: 'v_loss' reached 1.47123 (best 1.47123), saving model to '/home/dgxuser/Desktop/kailin/S91_no_mixing_transposed/FastPitch/2022-07-27_10-14-28/checkpoints/FastPitch--v_loss=1.4712-epoch=24.ckpt' as top 3
Epoch 41:  67%|██████▋   | 16/24 [00:10<00:05,  1.55it/s, loss=4.17, v_num=4-28]


In [39]:
def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        mixing_enabled, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"S{new_speaker_id}_{mixing}_transposed"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [40]:
speaker_id = '91'
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_transposed/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS91_transposed.json'

n = 0
with open(manifest_path, 'w') as fout: 
    with open(text_dir, "r") as f: 
        while True: 
          text = f.readline().strip()
          if text == '': 
            break 
          spec, audio = infer(spec_model, vocoder, text)
          audio = ipd.Audio(audio, rate=22050)
          audio = AudioSegment(audio.data)
          file_dir = audio_dir + str(n) + '.wav'
          audio.export(file_dir, format="wav")

          duration = librosa.core.get_duration(filename=file_dir)
          metadata = {
            "audio_filepath": file_dir,
            "duration": duration, 
            "text": text
          }
          json.dump(metadata, fout)
          fout.write('\n')   
          n = n + 1 
            
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()
        
error = wer(transcript, predicted)
print('Word Error Rate (After Pitch Peak Reduction): ' + str(error))

S91_no_mixing_transposed/FastPitch/2022-07-27_10-14-28/checkpoints/FastPitch--v_loss=1.4712-epoch=24-last.ckpt
[NeMo I 2022-07-27 10:24:58 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-27 10:25:33 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-27 10:25:33 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_transposed_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null


[NeMo I 2022-07-27 10:25:33 features:200] PADDING: 1
[NeMo I 2022-07-27 10:25:33 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-27 10:25:33 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-27 10:25:33 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-27 10:25:37 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-27 10:25:37 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-27 10:25:37 features:200] PADDING: 0


...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Word Error Rate (After Pitch Peak Reduction): 0.2730120062641378


# Testing with Multiple Speakers (2F1M)

In [41]:
def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        mixing_enabled, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"S{new_speaker_id}_{mixing}"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [46]:
## Finding original WER (individual speakers)
speaker_id = 208
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S208/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS208.json'

n = 0
with open(manifest_path, 'w') as fout: 
    with open(text_dir, "r") as f: 
        while True: 
          text = f.readline().strip()
          if text == '': 
            break 
          spec, audio = infer(spec_model, vocoder, text)
          audio = ipd.Audio(audio, rate=22050)
          audio = AudioSegment(audio.data)
          file_dir = audio_dir + str(n) + '.wav'
          audio.export(file_dir, format="wav")

          duration = librosa.core.get_duration(filename=file_dir)
          metadata = {
            "audio_filepath": file_dir,
            "duration": duration, 
            "text": text
          }
          json.dump(metadata, fout)
          fout.write('\n')   
          n = n + 1 
            
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()
        
error = wer(transcript, predicted)
print('Word Error Rate (Speaker 208): ' + str(error))

S208_no_mixing/FastPitch/2022-07-12_09-44-11/checkpoints/FastPitch--v_loss=1.0879-epoch=49-last.ckpt
[NeMo I 2022-07-27 10:43:41 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-27 10:44:15 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-27 10:44:15 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S208_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_

[NeMo I 2022-07-27 10:44:15 features:200] PADDING: 1
[NeMo I 2022-07-27 10:44:16 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-27 10:44:16 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-27 10:44:16 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-27 10:44:19 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-27 10:44:19 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-27 10:44:19 features:200] PADDING: 0


...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Word Error Rate (Speaker 194): 0.3380894379676353


In [47]:
speaker_id = 245
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S245/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS245.json'

n = 0
with open(manifest_path, 'w') as fout: 
    with open(text_dir, "r") as f: 
        while True: 
          text = f.readline().strip()
          if text == '': 
            break 
          spec, audio = infer(spec_model, vocoder, text)
          audio = ipd.Audio(audio, rate=22050)
          audio = AudioSegment(audio.data)
          file_dir = audio_dir + str(n) + '.wav'
          audio.export(file_dir, format="wav")

          duration = librosa.core.get_duration(filename=file_dir)
          metadata = {
            "audio_filepath": file_dir,
            "duration": duration, 
            "text": text
          }
          json.dump(metadata, fout)
          fout.write('\n')   
          n = n + 1 
            
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()
        
error = wer(transcript, predicted)
print('Word Error Rate (Speaker 245): ' + str(error))

S245_no_mixing/FastPitch/2022-07-12_10-14-25/checkpoints/FastPitch--v_loss=0.9827-epoch=49-last.ckpt
[NeMo I 2022-07-27 11:15:03 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-27 11:15:36 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-27 11:15:36 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S245_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_

[NeMo I 2022-07-27 11:15:36 features:200] PADDING: 1


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Word Error Rate (Speaker 245): 0.20497650948320864


In [48]:
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S135_208_245_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S135_208_245_val.json \
  sup_data_path=./fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.01 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=./S135_208_245_no_mixing \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  trainer.check_val_every_n_epoch=20 \
  model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 \
  model.n_speakers=1 model.pitch_mean=121.9 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

/home/dgxuser/anaconda3/envs/test4/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
[NeMo W 2022-07-27 11:25:51 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-27 11:25:52 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
########################################

Validation: 0it [00:00, ?it/s]
Epoch 19: 100%|██████████| 50/50 [00:33<00:00,  1.51it/s, loss=5.28, v_num=5-53]
                                                                                Epoch 19, global step 940: 'v_loss' reached 2.28459 (best 2.28459), saving model to '/home/dgxuser/Desktop/kailin/S135_208_245_no_mixing/FastPitch/2022-07-27_11-25-53/checkpoints/FastPitch--v_loss=2.2846-epoch=19.ckpt' as top 3
Epoch 39:  94%|█████████▍| 47/50 [00:23<00:01,  2.01it/s, loss=5.25, v_num=5-53]
Validation: 0it [00:00, ?it/s]
Epoch 39: 100%|██████████| 50/50 [00:31<00:00,  1.60it/s, loss=5.25, v_num=5-53]
                                                                                Epoch 39, global step 1880: 'v_loss' reached 2.72624 (best 2.28459), saving model to '/home/dgxuser/Desktop/kailin/S135_208_245_no_mixing/FastPitch/2022-07-27_11-25-53/checkpoints/FastPitch--v_loss=2.7262-epoch=39.ckpt' as top 3
Epoch 59:  94%|██████████▎| 47/50 [00:22<00:01,  2.08it/s, loss=5.2, v_num=5-5

Epoch 179: 100%|█████████| 50/50 [00:31<00:00,  1.61it/s, loss=4.69, v_num=5-53]
                                                                                Epoch 179, global step 8460: 'v_loss' was not in top 3
Epoch 199:  94%|████████▍| 47/50 [00:23<00:01,  1.96it/s, loss=4.81, v_num=5-53]
Validation: 0it [00:00, ?it/s]
Epoch 199: 100%|█████████| 50/50 [00:31<00:00,  1.56it/s, loss=4.81, v_num=5-53]
                                                                                Epoch 199, global step 9400: 'v_loss' reached 2.04121 (best 2.04121), saving model to '/home/dgxuser/Desktop/kailin/S135_208_245_no_mixing/FastPitch/2022-07-27_11-25-53/checkpoints/FastPitch--v_loss=2.0412-epoch=199.ckpt' as top 3
Epoch 219:  94%|████████▍| 47/50 [00:23<00:01,  2.01it/s, loss=4.36, v_num=5-53]
Validation: 0it [00:00, ?it/s]
Epoch 219: 100%|█████████| 50/50 [00:31<00:00,  1.58it/s, loss=4.36, v_num=5-53]
                                                                                Epoch 2

In [49]:
from pydub import AudioSegment
speaker_id = '135_208_245'
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S135_208_245/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS135_208_245.json'

n = 0
with open(manifest_path, 'w') as fout: 
    with open(text_dir, "r") as f: 
        while True: 
          text = f.readline().strip()
          if text == '': 
            break 
          spec, audio = infer(spec_model, vocoder, text)
          audio = ipd.Audio(audio, rate=22050)
          audio = Audio0Segment(audio.data)
          file_dir = audio_dir + str(n) + '.wav'
          audio.export(file_dir, format="wav")

          duration = librosa.core.get_duration(filename=file_dir)
          metadata = {
            "audio_filepath": file_dir,
            "duration": duration, 
            "text": text
          }
          json.dump(metadata, fout)
          fout.write('\n')   
          n = n + 1 
            
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()
        
error = wer(transcript, predicted)
print('Word Error Rate (Combined): ' + str(error))

S135_208_245_no_mixing/FastPitch/2022-07-27_11-25-53/checkpoints/FastPitch--v_loss=2.2329-epoch=339-last.ckpt
[NeMo I 2022-07-27 13:54:07 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-27 13:54:39 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-27 13:54:39 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S135_208_245_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
  

[NeMo I 2022-07-27 13:54:39 features:200] PADDING: 1
[NeMo I 2022-07-27 13:54:39 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-27 13:54:39 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-27 13:54:39 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-27 13:54:43 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-27 13:54:43 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-27 13:54:43 features:200] PADDING: 0


...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Word Error Rate (Combined): 0.21472072385592483
